In [5]:
# first install the required packages:
!pip install wikipedia



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=b0aa4e569de94cd0dd2551a6268e7cc09835ba9da4a05e921f880852152c5b37
  Stored in directory: c:\users\vardm\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [7]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)

In [21]:
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [22]:
wiki.name

'wikipedia'

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
gemini_api = os.getenv('GEMINI_API_KEY')
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Jainism")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=gemini_api)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
texts = text_splitter.split_documents(docs)

In [24]:
vectorstore = FAISS.from_documents(
    documents=texts,
    embedding=embeddings
)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000259814D2CE0>, search_kwargs={})

In [25]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,'wikipedia',
                      description = "search for information in wikipedia about the Jainism religion")

In [26]:
retriever_tool.name

'wikipedia'

In [27]:
tool = [wiki,retriever_tool]

In [28]:
tool

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\open_AI_api\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='wikipedia', description='search for information in wikipedia about the Jainism religion', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002599528EC20>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000259814D2CE0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000002599528ED40>, retriever=VectorStoreRetriever(tags=['FAIS

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=gemini_api,temperature = 0)

In [30]:
### agents
from langchain.agents import initialize_agent, AgentType
agent = initialize_agent(
    tools=tool,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # This works with Gemini too
    verbose=True
)

C:\Users\vardm\AppData\Local\Temp\ipykernel_33644\1328992411.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [31]:
from langchain import hub
# it is use for loading the prompt from the hub or either we can  use our prompt by prompt template

In [32]:
prompt  = hub.pull("hwchase17/openai-functions-agent")

d:\open_AI_api\venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [33]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
 
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tool,prompt)

In [ ]:

## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tool,verbose=False)#verbode false means it not give process information
agent_executor

AgentExecutor(verbose=False, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Ta

In [42]:

agent_executor.invoke({"input":"who is the mother of Mahavira?"})

{'input': 'who is the mother of Mahavira?',
 'output': "I am sorry, I cannot fulfill this request. The Wikipedia page retrieved does not mention the name of Mahavira's mother."}